In [1]:
## reference https://huggingface.co/learn/nlp-course/en/chapter7/6?fw=pt#training-a-causal-language-model-from-scratch

In [2]:
#installing some libraries
!pip install datasets
!pip install --upgrade jupyter ipywidgets



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import torch, transformers
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import PreTrainedModel, PretrainedConfig
from transformers import AutoModel, AutoConfig,AutoModelForCausalLM,AutoModelForTokenClassification

import json
import torch.nn as nn
import torch.nn.functional as F
#from datasets import load_dataset
import pandas as pd, numpy as np , random
from torch import cuda
import datetime
import warnings,itertools
from torch.cuda.amp import autocast, GradScaler

# Ignore all warnings
warnings.filterwarnings('ignore')
#pip install transformers bitsandbytes>=0.39.0 -q
import zipfile,logging

## Global Variables for training. Use the cell below to determing the following:
- training batch size
- Training with random weights or not
- The replacement strategy for replacing the token categoty at runtime
- The # of epochs for training.

## Here is the description of the various tokens

- O	Outside of a named entity
- B-MISC	Beginning of a miscellaneous entity right after another miscellaneous entity
- I-MISC	Miscellaneous entity
- B-PER	Beginning of a person’s name right after another person’s name
- I-PER	Person’s name
- B-ORG	Beginning of an organization right after another organization
- I-ORG	organization
- B-LOC	Beginning of a location right after another location
- I-LOC	Location

In [4]:
#global params for training
# hardcoding the T dimension to 512 as the BERT model processes sequences of length 512
B,T = 128,512
epoch = 100
random_init_wts = True

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print(device)
else:
    device = 'cpu'
#print(device)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
#os.environ["MKL_DEBUG_CPU_TYPE"] = "5"
global_tr_loss = torch.inf
global_val_loss = torch.inf
ner_model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
#print(global_tr_loss)
model_path = os.path.join("model")
print(model_path)
value_to_find = 6
pre_train = True

cuda:0


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model


In [5]:
directory = os.path.join('.','data','unzip_text_10M')  # Replace with your directory path
directory

'./data/unzip_text_10M'

In [6]:
def read_text(directory):
    """This function looks at the specified directory and unzips the files, reads the text provided in the file and finallly concatenates all the text in a single flattended list"""
    directory = os.path.join('.','data','unzip_text_10M',str(directory))  # Replace with your directory path
    print(f"directory :{directory}")
    # List all files in the directory
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    print(f"files:{files}")
    text_content = []
    # Read each file
    total_lines = 0
    for filenum,filename in enumerate(files):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            text_content.append(text)
            print(f"the file:{filename} has been appeneded to the uber list and its length is {len(text_content)} ")
            
    
    flattened_list = ''.join(text_content)
    assert (len(flattened_list) == total_lines , f"Expected {len(flattened_list)} to be equal to {total_lines}" )
    
    return flattened_list

    

In [7]:
train_list = read_text("train_10M")


directory :./data/unzip_text_10M/train_10M
files:['switchboard.train', 'simple_wiki.train', 'open_subtitles.train', 'gutenberg.train', 'childes.train', 'bnc_spoken.train']
the file:switchboard.train has been appeneded to the uber list and its length is 1 
the file:simple_wiki.train has been appeneded to the uber list and its length is 2 
the file:open_subtitles.train has been appeneded to the uber list and its length is 3 
the file:gutenberg.train has been appeneded to the uber list and its length is 4 
the file:childes.train has been appeneded to the uber list and its length is 5 
the file:bnc_spoken.train has been appeneded to the uber list and its length is 6 


In [8]:
len(train_list)

54215049

In [9]:
chunks = len(train_list)//(B*T)
print(chunks)

827


In [10]:
from transformers import BertLMHeadModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("dslim/bert-base-NER",return_tensors = "pt" , truncate = True, return_overflowing_tokens=True , padding = False,)
enc_train = tokenizer(train_list)

Token indices sequence length is longer than the specified maximum sequence length for this model (15756896 > 512). Running this sequence through the model will result in indexing errors


In [11]:
comp_ratio = len(train_list)/len(enc_train['input_ids'])
comp_ratio

3.440718844625236

In [12]:
val_list = read_text("dev")
enc_val = tokenizer(val_list)

directory :./data/unzip_text_10M/dev
files:['switchboard.dev', 'simple_wiki.dev', 'open_subtitles.dev', 'gutenberg.dev', 'childes.dev', 'bnc_spoken.dev']
the file:switchboard.dev has been appeneded to the uber list and its length is 1 
the file:simple_wiki.dev has been appeneded to the uber list and its length is 2 
the file:open_subtitles.dev has been appeneded to the uber list and its length is 3 
the file:gutenberg.dev has been appeneded to the uber list and its length is 4 
the file:childes.dev has been appeneded to the uber list and its length is 5 
the file:bnc_spoken.dev has been appeneded to the uber list and its length is 6 


### our strategy for bacthed training is to divide the tokenized text into chunks. The idea is to store these chunks in a dataframe so that each row stores a B*T number of tokens.At traning time, we'll use a batch size of 1 and feed each row (B*T) to the model to simulate batched training

In [13]:
def get_df_from_list(enc , B= B, T = T, val= False):
    """This function splits the tokeinzed text into chunks of B*T where each row in the dataset becomes a batch for the model"""
    chunk_size = B*T
        
    long_list_inp = enc['input_ids']
    long_list_attention = enc['attention_mask']

    # Step 3: Split the list into chunks and pad the last chunk if necessary
    chunks_inp = [long_list_inp[i:i + chunk_size] for i in range(0, len(long_list_inp), chunk_size)]
    chunks_att = [long_list_attention[i:i + chunk_size] for i in range(0, len(long_list_attention), chunk_size)]
    df = pd.DataFrame({'input_ids': chunks_inp,'attention_mask':chunks_att})
    return df
    

In [14]:
df_train_temp = get_df_from_list(enc_train)
# Display the DataFrame
df_train_temp.head()
print(f"Length of the dataframe is = {len(df_train_temp)}")

Length of the dataframe is = 241


In [15]:
df_val_temp = get_df_from_list(enc_val)
# Display the DataFrame
df_val_temp.head()
print(f"Length of the VALDATION dataframe is = {len(df_val_temp)}")


Length of the VALDATION dataframe is = 245


In [16]:
def pad_df(df,eos_char = tokenizer.pad_token_id,B = B, T = T,val = False):
    """This function pads the rows with tokeizer's attribute to ensure that all the rows of the dataframe are of the same length
    Using this approach prevents any unnessary padding and a custom collate funtion for model training and reduces memory footrprint."""
    if val:
        B = 2*B
    for ind,row in df.iterrows():
        if len(row['input_ids']) != B*T :
            print(f"row = {ind} and input_id length = {len(row['input_ids'])}")
            print(f"row = {ind} and attention length = {len(row['attention_mask'])}")
            pad_len = B*T - len(row['input_ids'])
            print(f"padding the row index {ind} with {pad_len} character")
            row['input_ids'] = row['input_ids']+ [eos_char] * pad_len
            #attention mask should be padded to 0
            row['attention_mask'] = row['attention_mask']+ [0] * pad_len
            print("#### POST CONCAT####")
            print(f"row = {ind} and input_id length = {len(row['input_ids'])}")
            print(f"row = {ind} and attention length ={len(row['attention_mask'])}")
    return df


def verify_len(df,val = False, B= B):
    row_ind = []
    if val:
        B = 2*B
    for ind,row in df.iterrows():
        if len(row['input_ids']) != B*T :
            row_ind.append(ind)
        else:
            continue
    if len(row_ind) !=0:
        print("CONCATENATION Did not work")
    else:
        print("CONCATENATION worked")
        
            
        
    
        
        
   
   

In [17]:
df_train  = pad_df(df_train_temp)
verify_len(df_train)

row = 240 and input_id length = 28256
row = 240 and attention length = 28256
padding the row index 240 with 37280 character
#### POST CONCAT####
row = 240 and input_id length = 65536
row = 240 and attention length =65536
CONCATENATION worked


In [18]:
df_val  = pad_df(df_val_temp)
verify_len(df_val)

row = 244 and input_id length = 32623
row = 244 and attention length = 32623
padding the row index 244 with 32913 character
#### POST CONCAT####
row = 244 and input_id length = 65536
row = 244 and attention length =65536
CONCATENATION worked


In [19]:
df_train.head()

,input_ids,attention_mask
0,"[101, 138, 131, 146, 112, 182, 1612, 1152, 113...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[117, 1681, 11764, 5558, 119, 139, 131, 11205,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[117, 138, 131, 1103, 2548, 117, 138, 131, 146...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[1107, 20119, 119, 138, 131, 2048, 117, 1674, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[1107, 1103, 8851, 16174, 10840, 18847, 1105, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


### In the cells below we classify all the tokens in the training data with specific NER tags. The idea is to create a repository of tokens that can be used to replace tokens during training

In [20]:
def get_keys_from_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key

In [21]:
uber_token_list_input = np.concatenate(df_train['input_ids']).tolist()
uber_token_list_att = np.concatenate(df_train['attention_mask']).tolist()


In [22]:
def classify_token(df):
    """This function processes the tokenized dataframe and produces a dictionay of classified tokens.
    Each key in the dictionary corresponds to a class of NER supported by the NER model and the values are the input_ids for the tokens."""
    
    ner_model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
    ner_model = torch.compile(ner_model)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    ner_model = ner_model.to(device)
    label_list = ner_model.config.id2label
    id_list = ner_model.config.label2id
    #print(f"id_list = {id_list}")
    classified_tokens = {value: [] for value in label_list.values()}
    #print(f"at the very beginnning the classified_tokens is {classified_tokens}")
    for ind,row in df.iterrows():
        #print(f"ind = {ind}| row = {row}")
        input_ids = row['input_ids']
        attention_mask = row['attention_mask']
        input_ids = torch.tensor(input_ids).view(B,T)
        attention_mask = torch.tensor(attention_mask).view(B,T)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        input_ids = torch.squeeze(input_ids,dim =0)
        attention_mask = torch.squeeze(attention_mask,dim = 0)
        #print(f"before performing NER shapes-> input_ids ={input_ids.shape}| attention_mask = {attention_mask.shape}")
        max_id = torch.max(input_ids)
        #print(f"Max token ID: {max_id}, Vocab size: {ner_model.config.vocab_size}")
        assert max_id < ner_model.config.vocab_size, "Token ID out of range"
        #print(f"shape of input going into the model |input_ids = {input_ids.shape}| attention_mask = {attention_mask.shape}")
        with torch.no_grad():
            outputs = ner_model(input_ids, attention_mask=attention_mask)
            

        # Get the predictions
        predictions = torch.argmax(outputs.logits, dim=2)
        #print(f"predictions shape = {predictions.shape}")
        predictions = list(torch.squeeze(predictions.view(B*T), dim = 0))
        #print(f"post conversion to list . len of prediction = {len(predictions)}")
        
        #print(f"predictions are {predictions}")
        #handling the case where a single row of the dataframe may have duplicate tokens
        list_of_token = list(input_ids.view(B*T))
        #print(f"length of list of tokens = {len(list_of_token)}")
        #print(f"shape of pred = {predictions.shape }| shape of input = {list_of_token.shape}")
        counter = 0
        for idx,elem in enumerate(list_of_token):
            pred = predictions[idx].item()
            #print(f"for token # {idx} prediction is {pred}| token = {elem.item()}")
            key = get_keys_from_value(id_list, pred)
            #print(f"This token {elem.item()} is being added to the key:{key}")
            classified_tokens[key].append(elem.item())
            counter+=1
            
            
        
        #print(f"processed row - {ind} and the classified toekn dict is - {classified_tokens}")    
    
    #the dataframe has been processed, now remove duplicates from the classified tokens dict
    #print(f"before removing duplicates |counter = {counter}")
    len_ch = 0
    for key,val in classified_tokens.items():
        print(len(classified_tokens[key]))
        len_ch+= len(classified_tokens[key])
    #print(f"before removing duplicates |len_ch = {len_ch}")
    
    
    assert len_ch == counter, "something went wrong. TRY again"

          
    
    for key,val in classified_tokens.items():
        classified_tokens[key] = list(set(val)) 
    ner_model.to("cpu")
    return classified_tokens

    
    


    
    
    

In [23]:
if pre_train == False:
    classified_tokens = classify_token(df_train)
    
# classified tokens
    with open('classified_tokens.json', 'w') as f:
        json.dump(classified_tokens, f)
else:
    with open('classified_tokens.json', 'r') as f:
        classified_tokens = json.load(f)

In [24]:
uniq_token = 0
for key,val in classified_tokens.items():
    print(len(classified_tokens[key]))
    uniq_token+=len(classified_tokens[key])

print(uniq_token)
    


27108
354
416
730
272
934
1793
569
219
32395


## Define the model

In [25]:
# Test the tokenizer:
model_name = "bert-base-cased"
if random_init_wts:
    config = AutoConfig.from_pretrained(model_name,is_decoder=True,vocab_size = 32768)
    # Initialize the model with random weights
    model = AutoModelForCausalLM.from_config(config)
else:
    #model = AutoModelForCausalLM.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained("bert-base-cased",is_decoder=True)

model = torch.compile(model)


### Data loaders and Dataset for batched training. This is where we replace the tokens at runtime.


In [26]:
class dataset_pyt_train(Dataset):
    def __init__(self, df, B = B, T = T,value_to_find = value_to_find, ner_model = ner_model):
        self.df = df
        print(f"Value of B {B}")
    """This is where we replace the tokens at runtime. The idea is to classify each token based on the same NER model and for an index that is even, if there is a token/set of tokens that belong to the specified category we replace it with the other tokens from the traiing data """    
    def __getitem__(self, idx):
        id_list = ner_model.config.label2id
        ner_label = [key for key, val in id_list.items() if val == value_to_find][0]                                
        source_inp = self.df.iloc[idx]['input_ids']
        source_att= torch.tensor(self.df.iloc[idx]['attention_mask'],dtype = torch.long)
        #print(f"ner_label = {ner_label}")
        ner_model.to("cuda")
        source_inp = torch.tensor(source_inp).to("cuda").view(B,T)
        source_att = torch.tensor(source_att).to("cuda").view(B,T)
        #print(f"after view shape of the source_inp = {source_inp.shape}| source_att = {source_att.shape}")

        with torch.no_grad():
            outputs = ner_model(input_ids = source_inp, attention_mask= source_att)

        if idx %2 == 0:
            # Get the predictions and 
            predictions = torch.argmax(outputs.logits, dim=2)
            #print(f"predictions shape = {predictions.shape}")
            predictions = torch.squeeze(predictions, dim = 0)
            #print(f"predictions shape post squ= {predictions.shape}")
            #predictions = list(torch.squeeze(predictions.view(B*T), dim = 0))
            #print(f"inside even_index{idx} in data loader...")

            #code to check and replace the values
            #first count occurences:
            occurrences = torch.sum(predictions == value_to_find).item()
            #print(f"Number of occurrences of {value_to_find}: {occurrences}")
            #indices = (predictions == value_to_find).nonzero().flatten()
            #print(f"indices.numel() = {indices.numel()}")
            if occurrences > 0:
                #extract tokens from sample space
                orig_tensor = source_inp.clone()
                sample_token_space = classified_tokens[ner_label]
                #print(f"sample_token_space = {sample_token_space}")
                replacement_tok_ids = random.choices(sample_token_space ,k=  occurrences)
                #print(f"the token selected from the sample space = {replacement_tok_ids}")
                #now replace the input_ids  with the selected token ids
                mask = (predictions == value_to_find)
                source_inp[mask] = torch.tensor(replacement_tok_ids,dtype = source_inp.dtype,device = source_inp.device)
                #print(f" original tensor = {orig_tensor}")
                #print(f" replaced tensor = {source_inp}")
                
                compare_elem = (source_inp != orig_tensor)
                #print(f"number of elem replaced {torch.sum(compare_elem).item()}")
                
                input_id =   source_inp.view(B,T)    
                attention_mask = source_att.view(B,T)   
                del source_inp,source_att
                return input_id, attention_mask
            else:
                #print(f"even index {idx} but did not find a toekn element to replace numel = 0")
                input_id =   source_inp.view(B,T)    
                attention_mask = source_att.view(B,T)
                return input_id, attention_mask


        #this code executes when either the odd index is being processed or there is no element to replace it with.
        input_id =   source_inp.view(B,T)    
        attention_mask = source_att.view(B,T)
        #print(f" odd index {idx} inside dataloder|returning shape of input_id = {input_id.shape} | shape of attention  = {attention_mask.shape}")
        del source_inp,source_att
                
                
        return input_id, attention_mask
        
    def __len__(self):
        #return the length of the dataframe
        return len(self.df)

In [27]:
class dataset_pyt_val(Dataset):
    def __init__(self, df, B = B, T = T ):
        self.df = df
                                        
    def __getitem__(self, idx):
        input_id_temp = torch.tensor(self.df.iloc[idx]['input_ids'],dtype = torch.long)
        att_mask = torch.tensor(self.df.iloc[idx]['attention_mask'],dtype = torch.long)
        input_id =   input_id_temp.view(B,T)    
        attention_mask = att_mask.view(B,T)   
        
        return input_id, attention_mask
        
    def __len__(self):
        #return the length of the dataframe
        return len(self.df)

In [28]:
train_dataset = dataset_pyt_train(df_train)
val_dataset = dataset_pyt_val(df_val)

train_loader = DataLoader(train_dataset,batch_size = 1, shuffle = True , num_workers = 0, pin_memory = False)
val_loader = DataLoader(val_dataset,batch_size = 1, shuffle = True , num_workers = 0, pin_memory = False)



Value of B 128


In [29]:
a,b = train_dataset[60]

In [30]:
print(f"Length of the train loader is {len(train_loader)}")
print(f"Length of the val loader is {len(val_loader)}")
print(f"num_tokens= {B*T*len(train_loader)}| unique tokens = {uniq_token}")

Length of the train loader is 241
Length of the val loader is 245
num_tokens= 15794176| unique tokens = 32395


In [31]:
def write_file(log_message, model_name = model_name ,random_init_wts = random_init_wts,value_to_find = value_to_find ):
    current_datetime = datetime.datetime.now()
    # Extract date and time components
    current_date = str(current_datetime.date())
    log_file = model_name +'random_init_wts'+ '_'+str(random_init_wts)+'_' +'NER_'+ str(value_to_find)+"_" +current_date+'.log'
    print(f"*****LOGGING INFO IN {log_file}*********")
    filepath = os.path.join("model",log_file)
    logging.basicConfig(filename=filepath, 
                    filemode='a',  # Overwrite the log file each time
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    level=logging.DEBUG)
    logger = logging.getLogger()
    logger.info(log_message)
    
    

    
    

In [32]:
@torch.no_grad
def eval_model(val_loader, model, epoch , device = device,tokenizer = tokenizer):
    global global_val_loss
    model.eval()
    model.to(device)
    e = epoch+1
    val_loss_accum = 0.0
    print(f"inside validation data for epoch {e}")
    #y_hat_val_list = []
    #y_val_list = []
    for ind,(input_id,attention_mask) in enumerate(val_loader):
        #print(f"id_list{id_list}")
        ids = torch.squeeze(input_id.to(device=device, non_blocking=True),dim = 0)
        att_mask = torch.squeeze(attention_mask.to(device=device, non_blocking=True),dim = 0)
        labels = ids
        #predictions
        #print(f"input_ids device = {input_ids.device}")
        with autocast(dtype = torch.bfloat16):
            model_output = model(input_ids = ids ,attention_mask = att_mask, labels = labels)
            input_embeddings = model.bert.embeddings(ids)
            logits = model_output.logits
            predictions = torch.argmax(logits, dim=-1)
            predicted_embeddings = model.bert.embeddings.word_embeddings(predictions)
            cos_sim = F.cosine_similarity(torch.squeeze(predicted_embeddings,dim = 0), torch.squeeze(input_embeddings,dim = 0), dim=1)
            cos_loss = 1- cos_sim.mean()
        val_loss_accum+=cos_loss.detach().item()
        del ids,att_mask,labels,cos_loss,input_embeddings,logits,predictions,predicted_embeddings
    
    if val_loss_accum < global_val_loss:
        print(f"Val loss has decreased -->reducing the global validation loss from {global_val_loss:.2f} to {val_loss_accum:.2f}")
        s1 = f"Val loss has decreased -->reducing the global validation loss from {global_val_loss:.2f} to {val_loss_accum:.2f}"
        global_val_loss = val_loss_accum
        print(f" validation loss for epoch = {e} is {val_loss_accum:.4f}")
        s2 = f" validation loss for epoch = {e} is {val_loss_accum:.4f}"
        #print metrics and save the model
        #y_hat_val = torch.cat(y_hat_val_list)
        #y_val = torch.cat(y_val_list)
        #acc_val = accuracy_score(y_val.cpu().numpy(), y_hat_val.cpu().numpy())
        #f1_val = f1_score(y_val.cpu().numpy(), y_hat_val.cpu().numpy(), average='micro')
        print(f" epoch= {e} :  val loss is {val_loss_accum:.4f} ")
        s3 = f" epoch= {e} :  val loss is {val_loss_accum:.4f} "
        #save the model
        
        # Get the current date and time
        current_datetime = datetime.datetime.now()
        # Extract date and time components
        current_date = str(current_datetime.date())
        current_time = str(current_datetime.time()).split('.')[0]
        file_name = 'model'+ current_date+current_time+'.pth'
        path = os.path.join("model",file_name)
        print(f"saving the model {file_name}")
        s4 = f"saving the model {file_name}"
        #torch.save(model.state_dict(), path)
        model.save_pretrained(path)
        tokenizer.save_pretrained(path)
        log_message = s1+s2+s3+s4
        write_file(log_message)
        
        #plot_confusion_matrix(y_val.cpu().numpy(), y_hat_val.cpu().numpy(), labels)
    else:
        print(f"No improvement in validation loss-->epoch= {e} and global val loss is {global_val_loss:.2f}")
        
    
    
    


In [33]:
def train_model(train_loader,val_loader,model,num_epoch = 100,device = device,tokenizer = tokenizer):
    global global_tr_loss
    model.train()
    device = device
    lr_custom = 5e-5
    print(f"inside train model. Device = {device}")
    #adding betas params per the paper
    model.to(device)
    optimizer = torch.optim.AdamW(params =  model.parameters(), lr= lr_custom,fused = True ,weight_decay = .1)
    extra_train = .1*num_epoch
    max_train_steps = int(num_epoch +extra_train )
    import time
    from transformers import get_linear_schedule_with_warmup
    scheduler_cos = transformers.get_cosine_schedule_with_warmup( optimizer= optimizer, num_warmup_steps =num_epoch*.1 ,num_training_steps= num_epoch-1 ,last_epoch = -1 )
    
        
    for i in range (max_train_steps):
        epoch_start_time = time.time()
        epoch_train_loss = 0.0
        optimizer.zero_grad(set_to_none=True)
        for ind,(input_id,attention_mask) in enumerate(train_loader):
            if ind == int(len(train_loader)/2):
                batch_time = time.time()
                duration = batch_time - epoch_start_time
                print(f"executing epoch:{i+1}, it took {duration/60} mins from beginning of epoch till batch#{ind}")
            
            ids = torch.squeeze(input_id.to(device=device, non_blocking=True),dim = 0)
            att_mask = torch.squeeze(attention_mask.to(device=device, non_blocking=True),dim = 0)
            labels = ids
            
            with autocast(dtype = torch.bfloat16):
                model_output = model(input_ids = ids ,attention_mask = att_mask, labels = labels)
                input_embeddings = model.bert.embeddings(ids)
                logits = model_output.logits
                predictions = torch.argmax(logits, dim=-1)
                predicted_embeddings = model.bert.embeddings.word_embeddings(predictions)
                cos_sim = F.cosine_similarity(torch.squeeze(predicted_embeddings,dim = 0), torch.squeeze(input_embeddings,dim = 0), dim=1)
                cos_loss = 1- cos_sim.mean()
                        
            cos_loss.backward()
            epoch_train_loss += cos_loss.detach().item()
            
            norm = torch.nn.utils.clip_grad_norm(model.parameters() , 1.0)
            if i <= num_epoch:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
            else:
                optimizer_reduced_lr.step()
                optimizer_reduced_lr.zero_grad(set_to_none=True)
                #print(f"Gradient accum step = {ind}|batch_loss after averaging ={batch_loss} | epoch_train_loss = {epoch_train_loss}")
                #print(f"Gradient accum step = {ind}|batch_loss after averaging ={batch_loss} | epoch_train_loss = {epoch_train_loss}")
                                
            del ids,att_mask,labels,input_embeddings,logits,predictions,predicted_embeddings
            
                
        #batch processing complete 
        if i <= num_epoch:
            current_lr = scheduler_cos.get_last_lr()[0]
            scheduler_cos.step()
        if i >= num_epoch:
            optimizer_reduced_lr = torch.optim.AdamW(params =  model.parameters(), lr= current_lr ,fused = True , weight_decay=.1)
            optimizer_reduced_lr.zero_grad(set_to_none=True)
            scheduler_constant = transformers.get_constant_schedule_with_warmup( optimizer = optimizer_reduced_lr ,num_warmup_steps = 0, last_epoch = -1 )
            scheduler_constant.step()
                      
        #mean_loss = torch.mean(torch.tensor(epoch_train_loss))
        epoch_end_time = time.time()
        epoch_durn = (epoch_end_time - epoch_start_time)
        num_token = B*T*len(train_loader)
        if (epoch_train_loss < global_tr_loss) :
            print(f"training loss has decreased---> reducing the global loss from {global_tr_loss:.2f} to {epoch_train_loss:.2f} | throughput = {int(num_token/epoch_durn)} tokens/second | norm = {norm:.4f} | learning rate = {current_lr:.5e}")
            global_tr_loss = epoch_train_loss
            print(f" epoch= {i+1} and  train loss is {epoch_train_loss:.2f}")
            #printing training metrices
            #y_hat = torch.cat(y_hat_list)
            #y = torch.cat(label_list)
            #acc = accuracy_score(y.cpu().numpy(), y_hat.cpu().numpy())
            #f1 = f1_score(y.cpu().numpy(), y_hat.cpu().numpy(), average='micro')
            #checking validation metrices
            if (i%4 == 0):
                eval_model(val_loader, model, epoch = i , device = device,tokenizer = tokenizer)
            
        else:
            print(f"No improvement in training loss..the global training loss is -->{global_tr_loss:.2f} | epoch= {i+1} and mean train loss is {epoch_train_loss:.2f}")
            
        
        
    
    return model
        
            
            
    
    

In [34]:
tr_model = train_model(train_loader, val_loader, model =  model,tokenizer = tokenizer)

inside train model. Device = cuda:0
executing epoch:1, it took 2.160727560520172 mins from beginning of epoch till batch#120
training loss has decreased---> reducing the global loss from inf to 240.84 | throughput = 65563 tokens/second | norm = 0.1712 | learning rate = 0.00000e+00
 epoch= 1 and  train loss is 240.84
inside validation data for epoch 1
Val loss has decreased -->reducing the global validation loss from inf to 244.90
 validation loss for epoch = 1 is 244.9022
 epoch= 1 :  val loss is 244.9022 
saving the model model2024-07-0919:03:13.pth
[2024-07-09 19:03:13,306] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


df: /root/.triton/autotune: No such file or directory
df: /root/.triton/autotune: No such file or directory
/opt/conda/compiler_compat/ld: cannot find -laio
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible
*****LOGGING INFO IN bert-base-casedrandom_init_wts_True_NER_6_2024-07-09.log*********
executing epoch:2, it took 2.041041453679403 mins from beginning of epoch till batch#120
training loss has decreased---> reducing the global loss from 240.84 to 234.41 | throughput = 68553 tokens/second | norm = 0.2636 | learning rate = 5.00000e-06
 epoch= 2 and  train loss is 234.41
executing epoch:3, it took 1.8105709354082744 mins fr

## Use this section if you want a model with Random weights

In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
# Define the model name
model_name = "distilgpt2"
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load the model configuration
config = AutoConfig.from_pretrained(model_name)

# Initialize the model with random weights
model = AutoModelForCausalLM.from_config(config)

# Check the model
#print(model)



In [36]:
x = torch.tensor(df_train.iloc[178]['input_ids'], dtype = torch.long)
att = torch.tensor(df_train.iloc[178]['attention_mask'], dtype = torch.long)
random_out = model(input_ids = x.view(B,T) , attention_mask = att.view(B,T), labels =   x.view(B,T) )

In [37]:
random_out

CausalLMOutputWithCrossAttentions(loss=tensor(11.0106, grad_fn=<NllLossBackward0>), logits=tensor([[[-0.0498, -0.9324, -0.2187,  ...,  0.5968,  0.5536,  0.1929],
         [ 0.0800, -0.1316, -0.4538,  ...,  0.0937,  0.5137,  0.6481],
         [ 0.2463, -0.6282, -0.4709,  ..., -0.2604,  1.0328,  0.5293],
         ...,
         [ 0.1569, -0.4576,  0.7966,  ..., -0.5286, -0.2449,  0.0846],
         [-0.9015, -0.3686, -0.5205,  ..., -0.5369, -0.3147, -0.5839],
         [-0.2887, -0.3370, -0.1329,  ..., -0.8134,  0.2153, -0.0743]],

        [[-0.6002, -0.8839,  0.1810,  ...,  0.3636, -0.4353, -0.1730],
         [-0.2791, -1.0208, -0.4995,  ...,  0.0821,  0.4229, -0.6961],
         [-0.4201, -0.0290,  0.5920,  ..., -0.0110,  0.4874,  0.4077],
         ...,
         [-0.0810, -0.4841, -0.1962,  ..., -0.0203,  0.0882, -0.3773],
         [-0.1496, -0.5701, -0.7049,  ..., -0.1519, -0.2026, -0.0081],
         [-0.3377, -0.4031, -0.2537,  ..., -0.2720, -0.4693, -0.2658]],

        [[-0.8142, -0.590